In [4]:
import random
from typing import List, Dict, Any, Optional, Tuple
from pydantic import BaseModel, Field
from abstractions.goap.entity import Statement
from abstractions.goap.spatial import GridMap, Node, GameEntity, ActionResult,ActionsPayload, SummarizedActionPayload
from abstractions.goap.interactions import MoveStep, PickupAction, DropAction, OpenAction, CloseAction, UnlockAction, LockAction
from abstractions.goap.game.main import generate_dungeon
from abstractions.goap.agents import  AgentGoal, FakeLLM, LLMAgent, CharacterAgent, RunMetadata
import outlines


In [5]:
def source_node_at_00(source: Node, target: Optional[Node] = None) -> bool:
    print("Checking if source node at 0, 0", source.position)
    if source.node.position.value == (0, 0):
        print("Source node at 0, 0", source.node.position)
        return True
    return False

# Create the grid map and generate the dungeon
grid_map = GridMap(width=10, height=10)
grid_map.register_actions([MoveStep, PickupAction, DropAction, OpenAction, CloseAction, UnlockAction, LockAction])
room_width, room_height = 6, 6
character, door, key, treasure = generate_dungeon(grid_map, room_width, room_height)

# Generate the entity type map
grid_map.generate_entity_type_map()

# Create the agent (either FakeLLM or LLMAgent)
agent = FakeLLM(grid_map, character.id)
# model_path = "./phi-2.Q4_K_M.gguf"  # Replace with the path to your model
# agent = LLMAgent(grid_map, character.id, model_path)

# Create the CharacterAgent with the chosen agent
character_agent = CharacterAgent(grid_map, agent)

# Set the agent's metadata
metadata = RunMetadata(character_id=character.id, run_number=1)
character_agent.set_metadata(metadata)

# Set the agent's goals
goal_statement = Statement(callables=[source_node_at_00])  # Example goal: reach position (0, 0)
goal = AgentGoal(statement=goal_statement, priority=1)
character_agent.set_goals([goal])

# Get the initial observation
initial_observation = character_agent.get_current_observation()

# Run the agent
character_agent.run(initial_observation)


Updating block attributes... for door

--- Step: 0 ---
Generated Action: action_name='MoveStep' source_entity_type='Character' source_entity_position=(5, 1) source_entity_id=None source_entity_name=None source_entity_attributes={} target_entity_type='Floor' target_entity_position=(4, 1) target_entity_id=None target_entity_name=None target_entity_attributes={}
Applying 1 actions
Action Result:
  State Before: {'source': {'blocks_movement': False, 'blocks_light': False, 'health': 100, 'max_health': 100, 'attack_power': 10, 'can_act': True, 'position': (5, 1), 'inventory': []}, 'target': {'blocks_movement': False, 'blocks_light': False, 'material': '', 'position': (4, 1), 'inventory': []}}
  State After: {'source': {'blocks_movement': False, 'blocks_light': False, 'health': 100, 'max_health': 100, 'attack_power': 10, 'can_act': True, 'position': (4, 1), 'inventory': []}, 'target': {'blocks_movement': False, 'blocks_light': False, 'material': '', 'position': (4, 1), 'inventory': []}}
Check